# Instruction Tests

In [1]:
import serial
import time
import pyvisa 
import numpy as np
from IPython.display import Image, display
from KeysightScope import KeysightScope

# You will have to change this to whatever COM port the pico is assigned when
# you plug it in.
# On Windows you can open device manager and look at the 'Ports (COM & LPT)' dropdown
# the pico will show up as 'USB Serial Device'
SWEEP_PORT = 'COM3'
PRAWN_PORT = 'COM7'

MHZ = 1000000

# sweep_conn = None
sweep_conn = serial.Serial(SWEEP_PORT, baudrate = 152000, timeout = 0.1)

# prawn_conn = None
prawn_conn = serial.Serial(PRAWN_PORT, baudrate = 152000, timeout = 0.1)

## Helpers

In [2]:
class BinaryRoutines():
    """
    Class handler for `setb` command to send binary tables to the dds-sweeper
    """

    def __init__(self, conn):
        self.conn = conn
        self.mirror_channels = False
        self.debug_run = True
        self.MAX_SIZE = 256
        self.instructions = None
        self.timing_mode = None

    def send(self, command: str, echo = True):
        if command[-1] != '\n':
            command += '\n'

        self.conn.write(command.encode())
        if echo:
            print(self.catch_response())

    def catch_response(self):
        resp = self.conn.readline().decode().strip()
        return resp

    def assert_OK(self):
        resp = self.conn.readline().decode().strip()
        assert resp == 'ok', 'Expected "ok", received "%s"' % resp

    def debug_on(self):
        self.conn.write(b'debug on\n')
        self.assert_OK()

    def debug_off(self):
        self.conn.write(b'debug off\n')
        self.assert_OK()

    def set_mode(self, sweep_mode: int, timing_mode: int, num_channels: int):
        self.conn.write(b'reset\n')
        self.assert_OK()

        self.timing_mode = timing_mode
        self.conn.write(b'mode %d %d \n' % (sweep_mode, timing_mode))
        self.assert_OK()
        self.conn.write(b'setchannels %d\n' % num_channels)
        self.assert_OK()
        if self.debug_run == True:
            print('sending commands to %d channels' % num_channels)

    def allocate(self, start_address: int, instructions: np.ndarray):
        self.conn.write(b'setb %d %d\n' % (start_address, len(instructions)))
        response = self.conn.readline().decode()
        if not response.startswith('ready'):
            response += ''.join([r.decode() for r in self.conn.readlines()])
            raise Exception(f'setb command failed, response: {repr(response)}')
        if self.debug_run == True:
            print(f'Got response: {response}')

    def table_to_memory(self, instructions: np.ndarray):
        self.conn.write(instructions.tobytes())
        self.assert_OK(), 'table not written correctly'
        if self.debug_run == True:
            print('table written to memory')

    def end_table(self, instructions: np.ndarray):
        self.conn.write(b'set 4 %d\n' % len(instructions))
        self.assert_OK(), 'table not stopped correctly'
        self.instructions = instructions
        if self.debug_run == True:
            print('table end command sent')

    def start_routine(self, hwstart = False):

        if hwstart:
            self.conn.write(b'hwstart\n')
        else:
            self.conn.write(b'start\n')
        self.assert_OK(), 'table did not start'
        if self.debug_run == True:
            print('table executed')

    def read_table(self, indices = None):
        self.conn.write(b'readtable\n')
        for i in range(self.instructions.size + 2): # plus two to catch header and footer
            resp = self.catch_response()
            print(resp)

            if "End of" in resp:
                break
            elif "Cannot" in resp:
                break
            elif len(resp) == 0:
                break
            
    def get_memory_layout(self):
        self.conn.write(b'getmode\n')
        # self.assert_OK, 
        # self.assert_OK, 'something went wrong getting memory layout'
        mode = self.catch_response()
        channels = self.catch_response()
        memory_layout = self.catch_response()
        print(f"mode {mode}")
        print(channels)
        print(memory_layout)
        # self.assert_OK, 'something went wrong getting memory layout'

In [3]:
def sendp(command: str, echo = True) -> str:
    # pico is expecting a newline to end every command
    if command[-1] != '\n':
        command += '\n'

    resp = ''
    try:
        prawn_conn.write(command.encode())
        if echo:
            resp = prawn_conn.readlines()
            resp = "".join([s.decode() for s in resp])

    except Exception as e:
        print("Encountered Error: ", e)

    return resp

In [4]:
## slightly different logic than the ad9959.c funcs due to python vs c
def get_ftw(freq_out: float, freq_sys: float = 500 * MHZ):

    ftw = (freq_out / freq_sys) * 2**32
    
    return(int(ftw))

def get_pow(phase: float):
    
    pow = round(phase / 360 * (2**14 - 1))

    return pow

def get_asf(amp: float):
    
    asf = round(amp * 1024)

    return asf

In [5]:


f1 = get_ftw(freq_out = 100 * MHZ)
f2 = get_ftw(freq_out = 120 * MHZ)
f3 = get_ftw(freq_out = 110 * MHZ)

a1 = 1023
a2 = 500

p0 = 0
p1 = get_pow(phase = 90)
p2 = get_pow(phase = 180)
p3 = get_pow(phase = 270)

t = 2

## Step Mode, External Time, One Channel

In [ ]:

dt = np.dtype([('frequency', '<u4'), 
               ('amplitude', '<u2'), 
               ('phase', '<u2')])

# n = 16352 // 2 # want to send n instructions, but 2 distinct
n = 20 // 2 # want to send n instructions, but 2 distinct

arr = [(f2, a1, p1), (f3, a2, p1)] * n # repeat these 2 instructions n times
step_instructions = np.array(arr, dtype = dt)

In [ ]:
single_step_test = BinaryRoutines(sweep_conn)

In [ ]:
single_step_test.catch_response()

In [ ]:
single_step_test.set_mode(sweep_mode=0, timing_mode=0, num_channels=1)
single_step_test.allocate(start_address=0, instructions=step_instructions)
single_step_test.table_to_memory(instructions=step_instructions)
single_step_test.end_table(instructions=step_instructions)

single_step_test.start_routine(hwstart=True)

In [ ]:
sendp(f'set 0 0 4000 {2*n}\r\n')
print(sendp('start\r\n', echo=True))

In [ ]:
print(sendp('status\r\n', echo=True))

In [ ]:
single_step_test.send('numtriggers')


In [ ]:
# single_step_test.read_table()

In [ ]:
single_step_test.send('numtriggers')

## Step Mode, External Time, Two Channels

In [ ]:
two_channel_test = BinaryRoutines(sweep_conn)

dt2 = np.dtype([('frequency0', '<u4'), 
               ('amplitude0', '<u2'), 
               ('phase0', '<u2'),
               ('frequency1', '<u4'), 
               ('amplitude1', '<u2'), 
               ('phase1', '<u2')]
               )

n = 8473 # max
n = 8447 # draws

chann0_ins = (f3, a1, p1)
chann1_ins = (f2, a1, p1)
tup_in = chann0_ins + chann1_ins # table layout reads chan{n} then chan{n+1} before next instruction

arr = [tup_in] * n 
two_channel_ins = np.array(arr, dtype = dt2)
two_channel_ins.size

In [ ]:
two_channel_test = BinaryRoutines(sweep_conn)

two_channel_test.set_mode(sweep_mode=0, timing_mode=0, num_channels=2)
two_channel_test.allocate(start_address=0, instructions=two_channel_ins)
two_channel_test.table_to_memory(instructions=two_channel_ins)
two_channel_test.end_table(instructions=two_channel_ins)

two_channel_test.start_routine(hwstart=True)

In [ ]:
sendp(f'set 0 0 4000 {n}\r\n')
print(sendp('start\r\n', echo=True))

In [ ]:
print(sendp('status\r\n', echo=True))

In [ ]:
two_channel_test.send('numtriggers')

## Step Mode, External Time, Three Channels

In [ ]:
three_channel_test = BinaryRoutines(sweep_conn)

dt3 = np.dtype([('frequency0', '<u4'), 
               ('amplitude0', '<u2'), 
               ('phase0', '<u2'),
               ('frequency1', '<u4'), 
               ('amplitude1', '<u2'), 
               ('phase1', '<u2'),
               ('frequency2', '<u4'), 
               ('amplitude2', '<u2'), 
               ('phase2', '<u2')]
               )

n = 5714 # max
n = 5710 # draws

chann0_ins = (f3, a1, p1)
chann1_ins = (f2, a1, p1)
chann2_ins = (f1, a1, p1)

tup_in = chann0_ins + chann1_ins + chann2_ins # table layout reads chan{n} then chan{n+1} before next instruction

arr = [tup_in] * n 
three_channel_ins = np.array(arr, dtype = dt3)
three_channel_ins.size

In [ ]:
three_channel_test = BinaryRoutines(sweep_conn)

three_channel_test.set_mode(sweep_mode=0, timing_mode=0, num_channels=3)
three_channel_test.allocate(start_address=0, instructions=three_channel_ins)
three_channel_test.table_to_memory(instructions=three_channel_ins)
three_channel_test.end_table(instructions=three_channel_ins)

three_channel_test.start_routine(hwstart=True)

In [ ]:
sendp(f'set 0 0 4000 {n}\r\n')
print(sendp('start\r\n', echo=True))

In [ ]:
print(sendp('status\r\n', echo=True))

In [ ]:
three_channel_test.send('numtriggers')

## Step Mode, External Time, Four Channels

In [ ]:
four_channel_test = BinaryRoutines(sweep_conn)

dt4 = np.dtype([('frequency0', '<u4'), 
               ('amplitude0', '<u2'), 
               ('phase0', '<u2'),
               ('frequency1', '<u4'), 
               ('amplitude1', '<u2'), 
               ('phase1', '<u2'),
               ('frequency2', '<u4'), 
               ('amplitude2', '<u2'), 
               ('phase2', '<u2'),
               ('frequency3', '<u4'), 
               ('amplitude3', '<u2'), 
               ('phase3', '<u2')]
               )


n = 4311 # max

chann0_ins = (f3, a1, p1)
chann1_ins = (f2, a1, p1)
chann2_ins = (f1, a1, p1)
chann3_ins = (f3, a1, p1)

tup_in = chann0_ins + chann1_ins + chann2_ins + chann2_ins # table layout reads chan{n} then chan{n+1} before next instruction

arr = [tup_in] * n 
four_channel_ins = np.array(arr, dtype = dt4)
four_channel_ins.size

In [ ]:
four_channel_test = BinaryRoutines(sweep_conn)

four_channel_test.set_mode(sweep_mode=0, timing_mode=0, num_channels=4)
four_channel_test.allocate(start_address=0, instructions=four_channel_ins)
four_channel_test.table_to_memory(instructions=four_channel_ins)
four_channel_test.end_table(instructions=four_channel_ins)

four_channel_test.start_routine(hwstart=True)

In [ ]:
sendp(f'set 0 0 4000 {n}\r\n')
print(sendp('start\r\n', echo=True))

In [ ]:
print(sendp('status\r\n', echo=True))


In [ ]:
four_channel_test.send('numtriggers')

## Sweep Mode, External Time, N Channels

In [ ]:
sweep1_dt = [('start_amplitude0', '<u2'),
             ('stop_amplitude0', '<u2'),
             ('delta0', '<u2'),
             ('rate0', '<u1'),
             ('start_amplitude1', '<u2'),
             ('stop_amplitude1', '<u2'),
             ('delta1', '<u2'),
             ('rate1', '<u1'),
             ('start_amplitude2', '<u2'),
             ('stop_amplitude2', '<u2'),
             ('delta2', '<u2'),
             ('rate2', '<u1'),
             ('start_amplitude3', '<u2'),
             ('stop_amplitude3', '<u2'),
             ('delta3', '<u2'),
             ('rate3', '<u1')
             ]

In [ ]:
# n = 8464 // 2 # sweep ext timer 1 channel
n = 2169 # sweep ext timer 2 channel
num_channels = 4

d = 1
r = 1

chann0_ins = (a1, a2, d, r)
chann1_ins = (a1, a2, d, r)
chann2_ins = (a1, a2, d, r)
chann3_ins = (a1, a2, d, r)

tup_in = chann0_ins + chann1_ins + chann2_ins + chann3_ins
arr = [tup_in] * n
sweep1_instructions = np.array(arr, dtype = np.dtype(sweep1_dt[:4 * num_channels]))
sweep1_test = BinaryRoutines(sweep_conn)

sweep1_test.set_mode(sweep_mode=1, timing_mode=0, num_channels=num_channels)
sweep1_test.allocate(start_address=0, instructions=sweep1_instructions)
sweep1_test.table_to_memory(instructions=sweep1_instructions)
sweep1_test.end_table(instructions=sweep1_instructions)

sweep1_test.start_routine(hwstart=True)

In [ ]:
sendp(f'set 0 0 4000 {4*n}\r\n')
print(sendp('start\r\n', echo=True))

In [ ]:
print(sendp('status\r\n', echo=True))


In [ ]:
sweep1_test.send('numtriggers')


## Step Mode, Internal Time, One Channel

In [ ]:
# dt_int = [
#     ('frequency0', '<u4'), 
#     ('amplitude0', '<u2'), 
#     ('phase0', '<u2'),
#     ('time0', '<u4'),
#     ('frequency1', '<u4'), 
#     ('amplitude1', '<u2'), 
#     ('phase1', '<u2'),
#     ('time1', '<u4'),
#     ('frequency2', '<u4'), 
#     ('amplitude2', '<u2'), 
#     ('phase2', '<u2'),
#     ('time2', '<u4'),
#     ('frequency3', '<u4'), 
#     ('amplitude3', '<u2'), 
#     ('phase3', '<u2'),
#     ('time3', '<u4')]

In [ ]:
dt_int = np.dtype([('frequency', '<u4'), 
               ('amplitude', '<u2'), 
               ('phase', '<u2'), 
               ('time', '<u4')])

f1 = get_ftw(freq_out = 100 * MHZ)
f2 = get_ftw(freq_out = 120 * MHZ)
f3 = get_ftw(freq_out = 110 * MHZ)

a1 = 1023
a2 = 500

p1 = get_pow(phase = 90)
p2 = get_pow(phase = 180)
p3 = get_pow(phase = 270)

t = 2

arr = [
(f2, a1, p1, t),
]

arr_in = arr * (4998)

In [ ]:
internal_step_instructions = np.array(arr_in, dtype = dt_int)
internal_step_instructions

In [ ]:
internal_time_test = BinaryRoutines(sweep_conn)
internal_time_test.set_mode(sweep_mode=0, timing_mode=1, num_channels=1)
internal_time_test.allocate(start_address=0, instructions=internal_step_instructions)
internal_time_test.table_to_memory(instructions=internal_step_instructions)
internal_time_test.end_table(instructions=internal_step_instructions)

internal_time_test.start_routine()

In [ ]:
print(internal_time_test.send('numtriggers'))

## Step Mode, Internal Time, Two Channels

In [ ]:
dt_int = [
    ('frequency0', '<u4'), 
    ('amplitude0', '<u2'), 
    ('phase0', '<u2'),
    ('time0', '<u4'),
    ('frequency1', '<u4'), 
    ('amplitude1', '<u2'), 
    ('phase1', '<u2'),
    ('time1', '<u4')]


arr = [
(f2, a1, p1, t, f2, a1, p1, t),
(f2, a2, p1, t, f2, a2, p1, t)
]

arr_in = arr * (4998//2)

In [ ]:
internal_step_instructions = np.array(arr_in, dtype = dt_int)
internal_step_instructions

In [ ]:
internal_time_test = BinaryRoutines(sweep_conn)
internal_time_test.set_mode(sweep_mode=0, timing_mode=1, num_channels=2)
internal_time_test.allocate(start_address=0, instructions=internal_step_instructions)
internal_time_test.table_to_memory(instructions=internal_step_instructions)
internal_time_test.end_table(instructions=internal_step_instructions)

internal_time_test.start_routine()

## Step Mode, Internal Time, Three Channels

In [ ]:
dt_int = [
    ('frequency0', '<u4'), 
    ('amplitude0', '<u2'), 
    ('phase0', '<u2'),
    ('time0', '<u4'),
    ('frequency1', '<u4'), 
    ('amplitude1', '<u2'), 
    ('phase1', '<u2'),
    ('time1', '<u4'),
    ('frequency2', '<u4'), 
    ('amplitude2', '<u2'), 
    ('phase2', '<u2'),
    ('time2', '<u4')]
arr = [
(f2, a1, p1, t, f2, a1, p1, t, f2, a1, p1, t),
(f2, a1, p1, t, f2, a1, p1, t, f2, a1, p1, t),
]

arr_in = arr * (4998//2)
internal_step_instructions = np.array(arr_in, dtype = dt_int)
internal_time_test = BinaryRoutines(sweep_conn)
internal_time_test.set_mode(sweep_mode=0, timing_mode=1, num_channels=3)
internal_time_test.allocate(start_address=0, instructions=internal_step_instructions)
internal_time_test.table_to_memory(instructions=internal_step_instructions)
internal_time_test.end_table(instructions=internal_step_instructions)

internal_time_test.start_routine()

## Step Mode, Internal Time, Four Channels

In [ ]:
dt_int = [
    ('frequency0', '<u4'), 
    ('amplitude0', '<u2'), 
    ('phase0', '<u2'),
    ('time0', '<u4'),
    ('frequency1', '<u4'), 
    ('amplitude1', '<u2'), 
    ('phase1', '<u2'),
    ('time1', '<u4'),
    ('frequency2', '<u4'), 
    ('amplitude2', '<u2'), 
    ('phase2', '<u2'),
    ('time2', '<u4'),
    ('frequency3', '<u4'), 
    ('amplitude3', '<u2'), 
    ('phase3', '<u2'),
    ('time3', '<u4')]
arr = [
(f2, a1, p1, t, f2, a1, p1, t, f2, a1, p1, t, f2, a1, p1, t),
(f2, a1, p1, t, f2, a1, p1, t, f2, a1, p1, t, f2, a1, p1, t),
]

arr_in = arr * (3960//2)
internal_step_instructions = np.array(arr_in, dtype = dt_int)
internal_time_test = BinaryRoutines(sweep_conn)
internal_time_test.set_mode(sweep_mode=0, timing_mode=1, num_channels=4)
internal_time_test.allocate(start_address=0, instructions=internal_step_instructions)
internal_time_test.table_to_memory(instructions=internal_step_instructions)
internal_time_test.end_table(instructions=internal_step_instructions)

internal_time_test.start_routine()

In [ ]:
# prawn_conn.close()
# sweep_conn.close()

## Sweep Mode, Internal Time, N Channels

In [6]:
int_sweep_dt = [
    ('start_amplitude0', '<u2'),
    ('stop_amplitude0', '<u2'),
    ('delta0', '<u2'),
    ('rate0', '<u1'),
    ('time0', '<u4'),
    ('start_amplitude1', '<u2'),
    ('stop_amplitude1', '<u2'),
    ('delta1', '<u2'),
    ('rate1', '<u1'),
    ('time1', '<u4'),
    ('start_amplitude2', '<u2'),
    ('stop_amplitude2', '<u2'),
    ('delta2', '<u2'),
    ('rate2', '<u1'),
    ('time2', '<u4'),
    ('start_amplitude3', '<u2'),
    ('stop_amplitude3', '<u2'),
    ('delta3', '<u2'),
    ('rate3', '<u1'),
    ('time3', '<u4')
]

In [101]:
n = 1995
num_channels = 4

d = 1
r = 1

tup_in = ()
chann0_ins = (a1, a2, d, r, t)
chann1_ins = (a1, a2, d, r, t)
chann2_ins = (a1, a2, d, r, t)
chann3_ins = (a1, a2, d, r, t)
list_of_chans = [chann0_ins, chann1_ins, chann2_ins, chann3_ins]

for i in range(num_channels):
    tup_in += list_of_chans[i]
arr = [tup_in] * n
int_sweep_instructions = np.array(arr, dtype = np.dtype(int_sweep_dt[:5 * num_channels]))
int_sweep_test = BinaryRoutines(sweep_conn)

int_sweep_test.set_mode(sweep_mode=1, timing_mode=1, num_channels=num_channels)
int_sweep_test.allocate(start_address=0, instructions=int_sweep_instructions)
int_sweep_test.table_to_memory(instructions=int_sweep_instructions)
int_sweep_test.end_table(instructions=int_sweep_instructions)

int_sweep_test.start_routine()

sending commands to 4 channels
Got response: ready for 87780 bytes

table written to memory
table end command sent
table executed


In [102]:
int_sweep_test.send('numtriggers')

1995
